In [4]:
! git clone https://github.com/DS3001/group10/

fatal: destination path 'group10' already exists and is not an empty directory.


In [5]:
#loading packages
import pandas as pd
import numpy as np

In [6]:
pwd

'/sfs/qumulo/qhome/acb4rn/probability'

In [7]:
import os
os.chdir('/sfs/qumulo/qhome/acb4rn/group10/project_2')

In [8]:
#loading in training and testing data
train = pd.read_csv("./data/training_data.csv")
test = pd.read_csv("./data/testing_data.csv")
train.head()

,Unnamed: 0,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,2465,68685,Male,36.0,0,0,Yes,Govt_job,Urban,65.87,32.2,formerly smoked,0
1,4311,59058,Female,45.0,0,0,Yes,Govt_job,Rural,68.66,25.3,never smoked,0
2,2375,46068,Male,58.0,0,0,No,Self-employed,Rural,170.93,30.7,Unknown,0
3,5017,36837,Female,61.0,0,0,Yes,Self-employed,Urban,69.88,27.1,never smoked,0
4,753,30550,Female,78.0,0,0,No,Private,Urban,103.86,30.6,Unknown,0


In [9]:
train['ever_married'].unique()

array(['Yes', 'No'], dtype=object)

In [10]:
#changing ever_married to 0 (no) and 1 (yes)
train['ever_married'] = train['ever_married'].replace({"No":0, "Yes":1})
test['ever_married'] = test['ever_married'].replace({"No":0, "Yes":1})
print(train['ever_married'].unique())
test['ever_married'].unique()

[1 0]


array([1, 0])

In [11]:
# checking residence type variable values
train['Residence_type'].unique()

array(['Urban', 'Rural'], dtype=object)

In [12]:
# making residence type into a numerical dummy (0: Urban, 1:Rural)
train['Residence_type'] = train['Residence_type'].replace({"Urban":0, "Rural":1})
test['Residence_type'] = test['Residence_type'].replace({"Urban":0, "Rural":1})

In [13]:
print(train['Residence_type'].unique())
test['Residence_type'].unique()

[0 1]


array([0, 1])

In [14]:
#checking avg glucose level variable nan values
train['avg_glucose_level'].isna().sum()

0

In [15]:
#checking gender variable values
train['gender'].unique()

array(['Male', 'Female', 'Other'], dtype=object)

In [16]:
#changing gender values to numbers (0:male, 1:female, 2:other)
train['gender'] = train['gender'].replace({"Male":0, "Female":1, "Other":2})
test['gender'] = test['gender'].replace({"Male":0, "Female":1, "Other":2})
print(train['gender'].unique())
test['gender'].unique()

[0 1 2]


array([0, 1])

In [17]:
#checking missing values for bmi
train['bmi'].isna().sum()

159

In [18]:
#using average bmi values 
train['bmi'] = train['bmi'].fillna(train['bmi'].mean())
test['bmi'] = test['bmi'].fillna(test['bmi'].mean())

In [19]:
# checking all na values are gone
print(test['bmi'].isna().sum())
train['bmi'].isna().sum()

0


0

In [20]:
train['smoking_status'].value_counts()

never smoked       1505
Unknown            1241
formerly smoked     699
smokes              642
Name: smoking_status, dtype: int64

In [21]:
#transforming smoking status variable to numeric (0:never, 1:formerly, 2:smokes, 3:unknown)
train['smoking_status'] = train['smoking_status'].replace({"never smoked":0, "formerly smoked":1, "smokes":2, 
                                                          "Unknown":3})
test['smoking_status'] = test['smoking_status'].replace({"never smoked":0, "formerly smoked":1, "smokes":2, 
                                                          "Unknown":3})
print(train["smoking_status"].value_counts())
test['smoking_status'].value_counts()

0    1505
3    1241
1     699
2     642
Name: smoking_status, dtype: int64


0    387
3    303
1    186
2    147
Name: smoking_status, dtype: int64

In [22]:
# sectioning interested variables into the train and test sets being used in the model
X_train = train.drop(['stroke', 'id', "Unnamed: 0", 'work_type'], axis=1 )
y_train = train['stroke']
X_test = test.drop(['stroke', 'id', "Unnamed: 0", 'work_type'], axis=1 )
y_test = test['stroke']

In [23]:
# loading linear model packages 
from sklearn.linear_model import LinearRegression as lr

TSS = np.sum( (y_test - y_test.mean())**2 )
N_test = len(y_test)

In [24]:
#fitting model and making predictions for test data
reg = lr().fit(X_train, y_train)
y_hat = reg.predict(X_test)

#calculating r-squared
rsqu = reg.score(X_test, y_test)
print("R-Squared: ", rsqu )

#calculating RMSE
SSE_lm =np.sum((y_test-y_hat)**2)
RMSE_lm = (SSE_lm/N_test)**(1/2)
R2_lm = 1-SSE_lm/TSS
print('RMSE: ', RMSE_lm)


R-Squared:  0.07751055818418306
RMSE:  0.20708397403676543


New Linear Regression Using Expanded Polynomials

In [25]:
from sklearn.preprocessing import PolynomialFeatures
#creating a function for the expander process
def poly_expand (dat, deg):
  expander = PolynomialFeatures(degree = deg, include_bias=False)
  Z = expander.fit_transform(dat)
  names = expander.get_feature_names()
  zdf = pd.DataFrame(data=Z, columns=names)
  return zdf

#creating a function for the regression
def lin_reg (X_train, y_train, X_test, y_test):
  reg_all = lr().fit(X_train, y_train)
  y_hat = reg_all.predict(X_test)
  #r-squared value
  r_squ = reg_all.score(X_test, y_test)
  # Metrics:
  SSE_lm = np.sum( (y_test-y_hat)**2 )
  RMSE_lm = (SSE_lm/N_test)**(1/2)
  R2_lm = 1 - SSE_lm/TSS
  return r_squ, RMSE_lm

In [26]:
#running polynomial expander with a degree of 2
poly_expand(X_train, 2)
poly_expand(X_test, 2)
lin_reg(X_train, y_train, X_test, y_test)

(0.07751055818418306, 0.20708397403676543)

In [27]:
#running polynomial expander with a degree of 3
poly_expand(X_train, 3)
poly_expand(X_test, 3)
lin_reg(X_train, y_train, X_test, y_test)

(0.07751055818418306, 0.20708397403676543)